In [ ]:
!pip install biopython pandas

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB 640.0 kB/s eta 0:00:05
   -- ------------------------------------- 0.2/2.8 MB 1.8 MB/s eta 0:00:02
   -- ------------------------------------- 0.2/2.8 MB 1.8 MB/s eta 0:00:02
   ------ --------------------------------- 0.5/2.8 MB 2.5 MB/s eta 0:00:01
   ------- -------------------------------- 0.6/2.8 MB 2.9 MB/s eta 0:00:01
   ------------- -------------------------- 0.9/2.8 MB 3.4 MB/s eta 0:00:01
   --------------------------- ------------ 1.9/2.8 MB 6.1 MB/s eta 0:00:01
   -------------------------------- ------- 2.3/2.8 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 6.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from Bio import SeqIO
import re
from Bio import SeqIO
import pandas as pd

In [25]:
def load_fasta(fasta_file):
    """
    Load a FASTA file and return a list of SeqIO records.
    """
    return list(SeqIO.parse(fasta_file, "fasta"))

virulent_phage  = list(SeqIO.parse("datasets/deephage_lifestyle/Dataset-1_virulent.fasta", "fasta"))
temperate_phage = list(SeqIO.parse("datasets/deephage_lifestyle/Dataset-1_temperate.fasta", "fasta"))
virulent_phage_2  = list(SeqIO.parse("datasets/deephage_lifestyle/Dataset-2_virulent.fasta", "fasta"))
temperate_phage_2 = list(SeqIO.parse("datasets/deephage_lifestyle/Dataset-2_temperate.fasta", "fasta"))

host_chr_pvog_fragments = list(SeqIO.parse("datasets/refseq_simulated_metagenome/host_chr_pvog_fragments.fasta", "fasta"))
host_plasmid_pvog_fragments = list(SeqIO.parse("datasets/refseq_simulated_metagenome/host_plasmid_pvog_fragments.fasta", "fasta"))
phage_fragment_set_shuffled = list(SeqIO.parse("datasets/refseq_simulated_metagenome/phage_fragment_set_shuffled.fasta", "fasta"))
phage_fragment_set = list(SeqIO.parse("datasets/refseq_simulated_metagenome/phage_fragment_set.fasta", "fasta"))
host_chr_pvog_fragments = list(SeqIO.parse("datasets/refseq_simulated_metagenome/host_chr_pvog_fragments.fasta", "fasta"))
host_plasmid_pvog_fragments = list(SeqIO.parse("datasets/refseq_simulated_metagenome/host_plasmid_pvog_fragments.fasta", "fasta"))


In [33]:
virulent_phage

[SeqRecord(seq=Seq('gtcccgcccccataggacacgctaaccaaacttttaggactagccagcatatca...cct'), id='Viru_gi|199|ref|NC_011421|', name='Viru_gi|199|ref|NC_011421|', description='Viru_gi|199|ref|NC_011421| Bacillus phage SPO1 linear', dbxrefs=[]),
 SeqRecord(seq=Seq('ttaggccgcaagggccttcatagttttagcgatttgggaaacttcatcatcact...tta'), id='Viru_gi|1|ref|NC_012635|', name='Viru_gi|1|ref|NC_012635|', description='Viru_gi|1|ref|NC_012635| Enterobacteria phage RB51 linear', dbxrefs=[]),
 SeqRecord(seq=Seq('ttaggccgcaagggccttcatagttttagcgatttgggaaacttcatcatcact...tta'), id='Viru_gi|3|ref|NC_012638|', name='Viru_gi|3|ref|NC_012638|', description='Viru_gi|3|ref|NC_012638| Enterobacteria phage RB14 linear', dbxrefs=[]),
 SeqRecord(seq=Seq('ctcatagttcataaacaccgagttcccccctatggcctccttcagttccaccca...ccc'), id='Viru_gi|31|ref|NC_011534|', name='Viru_gi|31|ref|NC_011534|', description='Viru_gi|31|ref|NC_011534| Kluyvera phage Kvp1 linear', dbxrefs=[]),
 SeqRecord(seq=Seq('cagcacaggagcacacaccatgggtatcttccgttccgcatccaatg

In [32]:
virulent_phage_2

[SeqRecord(seq=Seq('GGAGCAGGGAGAGAAAGGGACGAAAAGAAAACTATTTCGCCCATATACGTTCGT...ACG'), id='Viru_1_NC_009760', name='Viru_1_NC_009760', description='Viru_1_NC_009760', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGACAGATATCCTTGAAAAATTTGGCAACCTGTTAAAAGAAGCTGCGACGCCG...ACG'), id='Viru_2_NC_022343', name='Viru_2_NC_022343', description='Viru_2_NC_022343', dbxrefs=[]),
 SeqRecord(seq=Seq('GTCTACCTACTAAAGTTGCAAATTATCGGCATCAAGTTTTGCCATTAAGAGCAT...TTG'), id='Viru_3_NC_001741', name='Viru_3_NC_001741', description='Viru_3_NC_001741', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGTCACACAACGATACCCCCGCAAATAACACTAACGGCATAGCCGACGTTTAC...GCC'), id='Viru_4_NC_012558', name='Viru_4_NC_012558', description='Viru_4_NC_012558', dbxrefs=[]),
 SeqRecord(seq=Seq('AATTCGGTTTTTTAAAATAATAATTCCACTTGAATTTGCTAATAAGTATGATAG...AGG'), id='Viru_5_NC_003793', name='Viru_5_NC_003793', description='Viru_5_NC_003793', dbxrefs=[]),
 SeqRecord(seq=Seq('AATTCGGTTTTTTAAAATAATTATTCCCCTTGAATTTGCTAATAAGTATGATAG...AGG'), id='Viru_6_NC_001365', name='Vi

In [26]:
def extract_ncbi_id(record_id: str) -> str:
    """
    Given a record.id like
      - 'gi|199|ref|NC_011421|'
      - 'NC_052458.1_|Mycobacterium_phage_Ringer…'
      - 'NZ_AP018772.1_Haemophilus_influenzae…'
    return the core accession, e.g. 'NC_011421', 'NC_052458.1', 'NZ_AP018772.1'.
    """
    # 1) GI|…|ref|ACCESSION|  
    m = re.search(r"ref\|([^|]+)\|", record_id)
    if m:
        return m.group(1)

    # 2) Two‑letter prefix + underscore + two‑letter prefix + digits + optional .version
    #    e.g. NZ_AP018772.1
    m = re.match(r"^([A-Z]{1,2}_[A-Z]{2}\d+(?:\.\d+)?)", record_id)
    if m:
        return m.group(1)

    # 3) Fallback: one‑ or two‑letter prefix + underscore + digits + optional .version
    #    e.g. NC_011421.1, AF_000789
    m = re.match(r"^([A-Z]{1,2}_\d+(?:\.\d+)?)", record_id)
    if m:
        return m.group(1)

    # 4) Last resort: take everything up to first underscore or pipe
    return re.split(r"[_\|]", record_id, 1)[0]

def numeric_id(acc: str) -> str:
    """
    Given an accession like 'NC_011421.1', 'AZ123456.2', or 'AF_000789',
    return just the digits, e.g. '011421', '123456', '000789'.
    """
    m = re.search(r"(\d+)", acc)
    return m.group(1) if m else ""

temparate_phage_id = [ numeric_id(extract_ncbi_id(rec.id)) for rec in temperate_phage]
virulent_phage_id = [numeric_id(extract_ncbi_id(rec.id)) for rec in virulent_phage]
temparate_phage_2_id = [ numeric_id(extract_ncbi_id(rec.id)) for rec in temperate_phage_2]
virulent_phage_2_id = [numeric_id(extract_ncbi_id(rec.id)) for rec in virulent_phage_2]

phage_fragment_set_shuffled_id = [numeric_id(extract_ncbi_id(rec.id)) for rec in phage_fragment_set_shuffled]
phage_fragment_set_id = [numeric_id(extract_ncbi_id(rec.id)) for rec in phage_fragment_set]
host_chr_pvog_fragments_id = [numeric_id(extract_ncbi_id(rec.id)) for rec in host_chr_pvog_fragments]
host_plasmid_pvog_fragments_id = [numeric_id(extract_ncbi_id(rec.id)) for rec in host_plasmid_pvog_fragments]



In [27]:
df_virulent_phage_id = pd.DataFrame(virulent_phage_id, columns=["id"])
df_temperate_phage_id = pd.DataFrame(temparate_phage_id, columns=["id"])
df_virulent_phage_2_id = pd.DataFrame(virulent_phage_2_id, columns=["id"])
df_temperate_phage_2_id = pd.DataFrame(temparate_phage_2_id, columns=["id"])

df_phage_fragment_set_shuffled_id = pd.DataFrame(phage_fragment_set_shuffled_id, columns=["id"])
df_phage_fragment_set_id = pd.DataFrame(phage_fragment_set_id, columns=["id"])
df_host_chr_pvog_fragments_id = pd.DataFrame(host_chr_pvog_fragments_id, columns=["id"])
df_host_plasmid_pvog_fragments_id = pd.DataFrame(host_plasmid_pvog_fragments_id, columns=["id"])

In [28]:
df_virulent_phage_2_id, df_temperate_phage_2_id

(     id
 0      
 1      
 2      
 3      
 4      
 ...  ..
 1206   
 1207   
 1208   
 1209   
 1210   
 
 [1211 rows x 1 columns],
     id
 0     
 1     
 2     
 3     
 4     
 ..  ..
 424   
 425   
 426   
 427   
 428   
 
 [429 rows x 1 columns])

In [10]:
df_virulent_phage_id, df_host_chr_pvog_fragments_id

(        id
 0   011421
 1   012635
 2   012638
 3   011534
 4   012697
 ..     ...
 72  007021
 73  001909
 74  005083
 75  001902
 76  005282
 
 [77 rows x 1 columns],
             id
 0       018772
 1       018772
 2       018772
 3       018772
 4       018772
 ...        ...
 103998  015946
 103999  015946
 104000  015946
 104001  015946
 104002  015946
 
 [104003 rows x 1 columns])

In [20]:
def counts(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a DataFrame with a column 'id', return a new DataFrame with
    the counts of each unique id.
    """
    counts = df["id"].value_counts()
    df = df.groupby("id").size().reset_index(name="counts")
    return df

df_virulent_phage_id = counts(df_virulent_phage_id)
df_temperate_phage_id = counts(df_temperate_phage_id)
df_virulent_phage_2_id = counts(df_virulent_phage_2_id)
df_temperate_phage_2_id = counts(df_temperate_phage_2_id)

df_phage_fragment_set_shuffled_id = counts(df_phage_fragment_set_shuffled_id)
df_phage_fragment_set_id = counts(df_phage_fragment_set_id)
df_host_chr_pvog_fragments_id = counts(df_host_chr_pvog_fragments_id)
df_host_plasmid_pvog_fragments_id = counts(df_host_plasmid_pvog_fragments_id)

In [21]:
df_phage_fragment_set_id, df_phage_fragment_set_shuffled_id

(         id  counts
 0    047712      24
 1    047715      24
 2    047716      19
 3    047717      21
 4    047724      17
 ..      ...     ...
 883  055917       2
 884  055918       7
 885  055919      10
 886  055920       5
 887  055921       7
 
 [888 rows x 2 columns],
          id  counts
 0    047712      24
 1    047715      24
 2    047716      19
 3    047717      21
 4    047724      17
 ..      ...     ...
 883  055917       2
 884  055918       7
 885  055919      10
 886  055920       5
 887  055921       7
 
 [888 rows x 2 columns])

In [22]:
df_virulent_phage_id, df_temperate_phage_id

(        id  counts
 0   000866       1
 1   001271       1
 2   001418       1
 3   001420       1
 4   001421       1
 ..     ...     ...
 72  012696       1
 73  012697       1
 74  012740       1
 75  012741       1
 76  084414       1
 
 [77 rows x 2 columns],
          id  counts
 0    000871       1
 1    000872       1
 2    000896       1
 3    000902       1
 4    000924       1
 ..      ...     ...
 143  011976       1
 144  013055       1
 145  013059       1
 146  503408       1
 147  527608       1
 
 [148 rows x 2 columns])

In [23]:
df_virulent_phage_2_id, df_temperate_phage_2_id

(  id  counts
 0       1211,
   id  counts
 0        429)

In [14]:
df_host_chr_pvog_fragments_id, df_host_plasmid_pvog_fragments_id


(         id  counts
 0    015946     292
 1    018772     231
 2    018773     226
 3    018778     207
 4    019398     781
 ..      ...     ...
 224  890384     641
 225  890523     432
 226  890524     357
 227  890751     596
 228  992491     115
 
 [229 rows x 2 columns],
          id  counts
 0    009405       6
 1    015351      17
 2    016028      22
 3    020995       2
 4    021002       6
 ..      ...     ...
 158  883010       5
 159  890459      27
 160  890617      27
 161  890618       5
 162  890737      27
 
 [163 rows x 2 columns])

(        id  counts
 0   000866       1
 1   001271       1
 2   001418       1
 3   001420       1
 4   001421       1
 ..     ...     ...
 72  012696       1
 73  012697       1
 74  012740       1
 75  012741       1
 76  084414       1
 
 [77 rows x 2 columns],
          id  counts
 0    000871       1
 1    000872       1
 2    000896       1
 3    000902       1
 4    000924       1
 ..      ...     ...
 143  011976       1
 144  013055       1
 145  013059       1
 146  503408       1
 147  527608       1
 
 [148 rows x 2 columns])